In [1]:
# CUDA 11.2
# !unzip SEMEVAL-2014-REST.zip
# !python -m pip install paddlepaddle-gpu==2.3.2 -i https://pypi.tuna.tsinghua.edu.cn/simple
# !python -m pip install paddlenlp==2.4.2 -f https://www.paddlepaddle.org.cn/whl/windows/mkl/avx/stable.html

In [1]:
import warnings
from functions import *
from bert_attention import *
from bert_bare import *
from bert_lstm import *
import configparser
import os
from visualdl import LogWriter
from paddlenlp.transformers import SkepTokenizer, SkepModel, RobertaTokenizer, RobertaModel, BertTokenizer, BertModel
try:
    import seaborn as sns
except:
    pass

warnings.filterwarnings("ignore")
paddle.set_device("gpu:0")

config = configparser.ConfigParser()
config.read("config.config", encoding="utf-8")

Root = config.get('Settings','Root')
Num_attention_heads = int(config.get('Settings','Num_attention_heads'))
Num_epoch = int(config.get('Settings','Num_epoch'))
Batch_size = int(config.get('Settings','Batch_size'))
if '2014' in Root:
    Num_sentis = 4
else:
    Num_sentis = 3
Convert = config.get('Settings','Convert')
Max_seq_len = int(config.get('Settings','Max_seq_len'))
Q_length = int(config.get('Settings','Q_length'))
Learning_rate = float(config.get('Settings','Learning_rate'))
Model_name = config.get('Settings','Model_name')
Lstm = config.get('Settings','Lstm')
Res = config.get('Settings','Res')
Batch_func = config.get('Settings','Batch_func')
Loss_func = config.get('Settings','Loss_func')

label2id = {val:i for i, val in enumerate(open(Root+'label.txt', 'r', encoding='utf-8').read().splitlines())}
id2label = {i:val for i, val in enumerate(open(Root+'label.txt', 'r', encoding='utf-8').read().splitlines())}

if 'uncased' in Model_name or 'bert-base-chinese' in Model_name:
    model = BertModel.from_pretrained(Model_name);Tokenizer = BertTokenizer.from_pretrained(Model_name)
elif 'skep' in Model_name:
    model = SkepModel.from_pretrained(Model_name);Tokenizer = SkepTokenizer.from_pretrained(Model_name)

Hidden_size = model.config['hidden_size']

Convert_func =  convert_example_to_feature_normal if Convert == 'Normal' else convert_example_to_feature_special
Checkpoint = '_'.join([str(each) for each in ['cp',
                                              Root+Model_name+'/',
                                              Convert,
                                                Lstm,
                                                Res,
                                                Batch_func,
                                                Num_attention_heads,
                                                Loss_func
                                              ]])
Writer = LogWriter(logdir=Checkpoint)
os.mkdir(Checkpoint) if not os.path.exists(Checkpoint) else None
read_func = file_read_meituan if Root == '美团中文数据集/' else file_read_semeval
 
print(label2id)
print(Root,Num_sentis)
print(Checkpoint)
for each in config.items('Settings'):
    print('\t',each[0],'\t',each[1])
Train_loader,Dev_loader,Test_loader,Pred_loader = create_dataloader(Tokenizer,read_func,Convert_func,Root,Batch_size,['train','dev','test_','pred'])

[2023-12-27 22:21:31,070] [    INFO] - Already cached C:\Users\93461\.paddlenlp\models\skep_ernie_1.0_large_ch\skep_ernie_1.0_large_ch.vocab.txt
[2023-12-27 22:21:31,108] [    INFO] - tokenizer config file saved in C:\Users\93461\.paddlenlp\models\skep_ernie_1.0_large_ch\tokenizer_config.json
[2023-12-27 22:21:31,112] [    INFO] - Special tokens file saved in C:\Users\93461\.paddlenlp\models\skep_ernie_1.0_large_ch\special_tokens_map.json
[2023-12-27 22:21:31,115] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/community/skep_ernie_1.0_large_ch\vocab.txt and saved to C:\Users\93461\.paddlenlp\models\skep_ernie_1.0_large_ch
[2023-12-27 22:21:31,117] [    INFO] - Found C:\Users\93461\.paddlenlp\models\skep_ernie_1.0_large_ch\vocab.txt
[2023-12-27 22:21:31,120] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/community/skep_ernie_1.0_large_ch\added_tokens.json and saved to C:\Users\93461\.paddlenlp\models\skep_ernie_1.0_large_ch
[2023-12-27 22:21:31,121] [  

{'舒适': 0, '耐久': 1, '精细': 2, '保障': 3, '牌证': 4, '外观': 5, '外部': 6, '响应及时性': 7, '沟通有效性': 8, '信息安全性': 9, '补偿合理性': 10, '服务专业性': 11}
感知质量数据集/ 3
cp_感知质量数据集/skep_ernie_1.0_large_ch/_Normal_Lstm_Res_max_8_cross_entropy
	 q_length 	 20
	 max_seq_len 	 100
	 root 	 感知质量数据集/
	 model_name 	 skep_ernie_1.0_large_ch
	 num_epoch 	 15
	 batch_size 	 4
	 convert 	 Normal
	 learning_rate 	 2e-05
	 num_attention_heads 	 8
	 batch_func 	 max
	 loss_func 	 cross_entropy
	 lstm 	 Lstm
	 res 	 Res
读取文件路径： 感知质量数据集/关键词抽取结果_train.csv
读取文件路径： 感知质量数据集/关键词抽取结果_dev.csv
读取文件路径： 感知质量数据集/关键词抽取结果_test_.csv
读取文件路径： 感知质量数据集/关键词抽取结果_pred.csv


## BERT_Attention

In [3]:
# 训练与测试
from tqdm import tqdm
Train_loader,Dev_loader,Test_loader,Pred_loader = create_dataloader(Tokenizer,read_func,Convert_func,Root,Batch_size,['train','dev','test_','pred'])
Lr_scheduler = paddle.optimizer.lr.LinearWarmup(learning_rate=Learning_rate,warmup_steps=int(Num_epoch * 0.1 * len(Train_loader)),start_lr=0,end_lr=Learning_rate,verbose=False)
Model = Bert_LSTM_Attention(model,Hidden_size,len(label2id),Num_attention_heads,Num_sentis,Batch_func,Lstm,Res)
Optimizer = paddle.optimizer.AdamW(
    learning_rate=Lr_scheduler,
    parameters=Model.parameters(),
    weight_decay=0.01,
    apply_decay_param_fun=lambda x: x in [p.name for n, p in Model.named_parameters()if not any(nd in n for nd in ["bias", "norm"])])
# train_bert_attention(Model, Train_loader, Num_epoch,  Writer, Lr_scheduler, Optimizer, Checkpoint, 20, Max_seq_len, Num_sentis, Loss_func)

def test_bert_attention(Model, test_loader,  max_seqlength, checkpoint, num_sentis, root):
    Model.set_state_dict(paddle.load(f"{checkpoint}/best_cls_bert_attention.pdparams"))
    Model.eval()
    id2label = {i:val for i, val in enumerate(open(root+'label.txt', 'r', encoding='utf-8').read().splitlines())}
    with paddle.no_grad():
        Labels_3=[];Preds_3=[]  # 用于情感分类
        Labels_4=[];Preds_4=[]  # 用于维度分类
        Labels_5=[];Preds_5=[]  # 用于维度-情感分类
        S_G=0;len_S=0;len_G=0   # 用于维度-情感分类
        S_G2=0;len_S2=0;len_G2=0   # 用于维度-情感分类

        for batch_data in tqdm(test_loader()):
            input_ids_data, input_ids_term, attention_mask_input_data, attention_mask_input_term, senti_labels = batch_data
            state, outputs_bert, cross_attention = Model(input_ids_data,
                                                                  input_ids_term,
                                                                  attention_mask_input_data,
                                                                  attention_mask_input_term,
                                                                  max_seqlength)
            # 把模型输出转为独热的形式
            state2_ = paddle.reshape(state, [len(state), -1, num_sentis + 1])
            state2_ = paddle.nn.functional.softmax(state2_, axis=-1)
            # 对于每一个样本
            for each_sample in range(0,len(state2_)):

                state2_pred_3 = [];labels_3=[]  # 用于情感分类
                state2_pred_4 = [];labels_4=[]  # 用于维度分类
                state2_pred_5 = [];labels_5=[]  # 用于维度-情感分类

                # 对于每一个标签
                for i in range(0,len(state2_[each_sample])):
                    # 处理维度-情感分类，和维度分类
                    state = state2_[each_sample][i]

                    # 对于预测值的每一个维度，如果不是0，就把维度和情感值拼接起来
                    if int(paddle.argmax(state)) > 0:
                        state2_pred_5.append(str(int(paddle.argmax(state)))+'_'+id2label[i])
                        state2_pred_4.append(id2label[i])
                        len_S+=1;len_S2+=1

                    # 对于标签的每一个维度，如果不是0，就把维度和情感值拼接起来
                    if senti_labels[each_sample][i] > 0:
                        labels_5.append(str(int(senti_labels[each_sample][i]))+'_'+id2label[i])
                        labels_4.append(id2label[i])
                        len_G+=1;len_G2+=1

                    # 处理情感分类，如果预测和标签的维度都不是0，就把情感加入到预测值列表中
                    state2_pred_3.append(paddle.argmax(state)) if paddle.argmax(state) != 0 and senti_labels[each_sample][i] != 0 else None
                    labels_3.append(senti_labels[each_sample][i]) if paddle.argmax(state) != 0 and senti_labels[each_sample][i] != 0 else None

                Preds_3.extend(state2_pred_3);Labels_3.extend(labels_3)
                Preds_4.append(state2_pred_4);Labels_4.append(labels_4)
                Preds_5.append(state2_pred_5);Labels_5.append(labels_5)

        for each_sample in range(0,len(Preds_5)):
            interlist = list(set(Preds_5[each_sample]).intersection(set(Labels_5[each_sample])))
            for i in range(0,len(interlist)):
                S_G+=1

        for each_sample in range(0,len(Preds_4)):
            interlist = list(set(Preds_4[each_sample]).intersection(set(Labels_4[each_sample])))
            for i in range(0,len(interlist)):
                S_G2+=1

        P=S_G/len_S
        R=S_G/len_G
        F1=2*P*R/(P+R) if P+R != 0 else 0

        P2=S_G2/len_S2
        R2=S_G2/len_G2
        F12=2*P2*R2/(P2+R2) if P2+R2 != 0 else 0
        print(checkpoint)
        print('维度-情感分类效果\n','P:',str(P)[0:5],'R:',str(R)[0:5],'F1:',str(F1)[0:5],'S_G:',str(S_G)[0:5],'len_S:',len_S,'len_G:',len_G)
        print('维度-分类效果\n','P:',str(P2)[0:5],'R:',str(R2)[0:5],'F1:',str(F12)[0:5],'S_G:',str(S_G2)[0:5],'len_S:',len_S2,'len_G:',len_G2)
        print('情感-分类效果',classification_report(Labels_3, Preds_3, digits=4),sep='\n')

        data = np.zeros([3,3])
        for i in range(0,len(Labels_3)):
            data[Labels_3[i]-1,Preds_3[i]-1] += 1

        pd.DataFrame(data,columns=['1','2','3'],index=['1','2','3']).to_csv(checkpoint+'/confusion_matrix.csv')

test_bert_attention(Model, Test_loader, Max_seq_len, Checkpoint, Num_sentis, Root)

读取文件路径： 感知质量数据集/关键词抽取结果_train.csv
读取文件路径： 感知质量数据集/关键词抽取结果_dev.csv
读取文件路径： 感知质量数据集/关键词抽取结果_test_.csv
读取文件路径： 感知质量数据集/关键词抽取结果_pred.csv


100%|██████████| 142/142 [00:17<00:00,  8.16it/s]

cp_感知质量数据集/skep_ernie_1.0_large_ch/_Normal_Lstm_Res_max_8_cross_entropy
维度-情感分类效果
 P: 0.759 R: 0.712 F1: 0.734 S_G: 416 len_S: 548 len_G: 584
维度-分类效果
 P: 0.885 R: 0.830 F1: 0.856 S_G: 485 len_S: 548 len_G: 584
情感-分类效果
              precision    recall  f1-score   support

           1     0.9141    0.8098    0.8588       184
           2     0.7215    0.8382    0.7755       136
           3     0.9329    0.9273    0.9301       165

    accuracy                         0.8577       485
   macro avg     0.8562    0.8584    0.8548       485
weighted avg     0.8665    0.8577    0.8597       485



In [4]:
# 调整
Model.eval()
from tqdm import tqdm
Model.set_state_dict(paddle.load(f"{Checkpoint}/best_cls_bert_attention.pdparams"))
def pred_bert_attention_for_one_case(Model, tokenizer, sentence, max_seq_len, checkpoint, num_sentis, root):

    Model.eval()

    id2label = {i: val for i, val in enumerate(open(root + 'label.txt', 'r', encoding='utf-8').read().splitlines())}
    label2id = {val: i for i, val in enumerate(open(root + 'model_label_cls.dict', 'r', encoding='utf-8').read().splitlines())}
    id2label2 = {str(i): val for i, val in enumerate(open(root + 'model_label_cls.dict', 'r', encoding='utf-8').read().splitlines())}

    encoded_inputs_data = tokenizer(sentence.split(',')[2],max_length=max_seq_len,truncation=True,padding='max_length')
    encoded_inputs_term = tokenizer(sentence.split(',')[1],max_length=max_seq_len,truncation=True,padding='max_length')
    encoded_inputs_data['attention_mask']=[1 if i!=tokenizer.pad_token_id else 0 for i in encoded_inputs_data['input_ids']]
    encoded_inputs_term['attention_mask']=[1 if i!=tokenizer.pad_token_id else 0 for i in encoded_inputs_term['input_ids']]

    with paddle.no_grad():
        input_ids_data = paddle.to_tensor([encoded_inputs_data['input_ids']])
        input_ids_term = paddle.to_tensor([encoded_inputs_term['input_ids']])
        attention_mask_input_data = paddle.to_tensor([encoded_inputs_data['attention_mask']])
        attention_mask_input_term = paddle.to_tensor([encoded_inputs_term['attention_mask']])

        state, outputs_bert, cross_attention = Model(input_ids_data,
                                                     input_ids_term,
                                                     attention_mask_input_data,
                                                     attention_mask_input_term,
                                                     max_seq_len)

        state2_ = paddle.reshape(state, [len(state), -1, num_sentis + 1])
        state2_ = paddle.nn.functional.softmax(state2_, axis=-1)
        state2_ = state2_[0]
        # print(id2label2)
        with open(r'D:\python_common\代码_感知质量大论文\公开数据处理\cp_感知质量数据集\skep_ernie_1.0_large_ch\_Normal_Lstm_Res_max_8_cross_entropy\result_sc.tsv','a+',encoding='utf-8') as f:
            for i in range(0, len(state2_)):
                state = state2_[i]
                if int(paddle.argmax(state)) > 0:
                    if str(sentence.split(',')[0]) != str(label2id[id2label[i] + str(int(paddle.argmax(state)))]):

                        f.write(sentence.split(',')[1] + '\t' +
                                sentence.split(',')[2].strip() + '\t' +

                                sentence.split(',')[0] + '\t' +
                                str(id2label2[sentence.split(',')[0]]) + '\t' +

                                str(label2id[id2label[i] + str(int(paddle.argmax(state)))]) + '\t' +
                                id2label[i] + str(int(paddle.argmax(state))) + '\n')
with open('D:\python_common\代码_感知质量大论文\公开数据处理\感知质量数据集\关键词抽取结果_test_.csv','r',encoding='utf-8') as f:
    data = f.readlines()
for each in tqdm(data):
    pred_bert_attention_for_one_case(Model, Tokenizer, each, Max_seq_len, Checkpoint, Num_sentis, Root)



100%|██████████| 590/590 [00:50<00:00, 11.64it/s]


In [10]:
def test_bert_attention_for_one_case(Model, tokenizer, sentence, max_seq_len, checkpoint, num_sentis, root):
    Model.set_state_dict(paddle.load(f"{checkpoint}/best_cls_bert_attention.pdparams"))
    Model.eval()

    id2label = {i: val for i, val in enumerate(open(root + 'label.txt', 'r', encoding='utf-8').read().splitlines())}
    label2id = {val: i for i, val in enumerate(open(root + 'model_label_cls.dict', 'r', encoding='utf-8').read().splitlines())}

    encoded_inputs_data = tokenizer(sentence.split(',')[1],max_length=max_seq_len,truncation=True,padding='max_length')
    encoded_inputs_term = tokenizer(sentence.split(',')[0],max_length=max_seq_len,truncation=True,padding='max_length')
    encoded_inputs_data['attention_mask']=[1 if i!=tokenizer.pad_token_id else 0 for i in encoded_inputs_data['input_ids']]
    encoded_inputs_term['attention_mask']=[1 if i!=tokenizer.pad_token_id else 0 for i in encoded_inputs_term['input_ids']]

    with paddle.no_grad():
        input_ids_data = paddle.to_tensor([encoded_inputs_data['input_ids']])
        input_ids_term = paddle.to_tensor([encoded_inputs_term['input_ids']])
        attention_mask_input_data = paddle.to_tensor([encoded_inputs_data['attention_mask']])
        attention_mask_input_term = paddle.to_tensor([encoded_inputs_term['attention_mask']])

        state, outputs_bert, cross_attention = Model(input_ids_data,
                                                     input_ids_term,
                                                     attention_mask_input_data,
                                                     attention_mask_input_term,
                                                     max_seq_len)

        state2_ = paddle.reshape(state, [len(state), -1, num_sentis + 1])
        state2_ = paddle.nn.functional.softmax(state2_, axis=-1)
        state2_ = state2_[0]

        for i in range(0, len(state2_)):
            state = state2_[i]
            if int(paddle.argmax(state)) > 0:
                print('------------------')
                print(id2label[i]+str(int(paddle.argmax(state))))
                print(label2id[id2label[i]+str(int(paddle.argmax(state)))])
                print('------------------')

test_bert_attention_for_one_case(Model, Tokenizer, '好用非常,保暖性强！非常好用！', Max_seq_len, Checkpoint, Num_sentis, Root)


------------------
外部3
30
------------------


In [ ]:
for pinpai in ['ALBD','AND','AT','BNL','GRN','LN','QPL','TB']:
    Pred_loader = create_dataloader(Tokenizer,read_func,Convert_func,Root,Batch_size,None,pinpai,pred_only=True)
    pred_bert_attention(Model, Pred_loader, Max_seq_len, Checkpoint, Num_sentis, Root, pinpai)


## BERT_bare

In [4]:
Model = Bert_Bare(model,Hidden_size,len(label2id),Num_sentis)
Optimizer = paddle.optimizer.AdamW(
    learning_rate=Lr_scheduler,
    parameters=Model.parameters(),
    weight_decay=0.01,
    apply_decay_param_fun=lambda x: x in [p.name for n, p in Model.named_parameters()if not any(nd in n for nd in ["bias", "norm"])])
train_bert_bare(Model, Train_loader, Num_epoch,  Writer, Lr_scheduler, Optimizer, Checkpoint, 20, Max_seq_len, Num_sentis)
test_bert_bare(Model, Test_loader, Max_seq_len, Checkpoint, Num_sentis, Root)

TypeError: train_bert_bare() missing 1 required positional argument: 'loss_func'

## Bert_lstm

In [ ]:
Model = Bert_LSTM(model, Hidden_size, Num_sentis, len(label2id))
Optimizer = paddle.optimizer.AdamW(
    learning_rate=Lr_scheduler,
    parameters=Model.parameters(),
    weight_decay=0.01,
    apply_decay_param_fun=lambda x: x in [p.name for n, p in Model.named_parameters()if not any(nd in n for nd in ["bias", "norm"])])
train_bert_lstm(Model, Train_loader, Num_epoch,  Writer, Lr_scheduler, Optimizer, Checkpoint, 20,  Num_sentis, Loss_func)
test_bert_lstm(Model, Test_loader, Max_seq_len, Checkpoint, Num_sentis, Root)